In [10]:
import kaggle
import os
import zipfile

In [11]:
data_dir = "./Data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

Using here an initial dataset found at https://www.kaggle.com/datasets/paulchambaz/google-street-view/data

This will be replaced in the future

In [12]:
!kaggle datasets download -d paulchambaz/google-street-view -p {data_dir}

google-street-view.zip: Skipping, found more recently modified local copy (use --force to force download)


In [13]:
with zipfile.ZipFile(f"{data_dir}/google-street-view.zip", 'r') as zip_ref:
    zip_ref.extractall(data_dir)

In [14]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [15]:
df = pd.read_csv('./Data/dataset/coords.csv', names=['latitude', 'longitude'])
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))


Adding country labels for each datapoint. 

In [16]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf = gdf.set_crs(world.crs)
intersect_gdf = gpd.sjoin(gdf, world, how="left", op='intersects')
df['country'] = intersect_gdf['name']

<ipython-input-16-56dc7eecbd49>:1: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
c:\Users\Meau\Python\lib\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


Dropping countries not in the list at ./Data/country_list.csv (i.e. only keeping European countries)

In [17]:
country_list = pd.read_csv('./Data/country_list.csv')
european = country_list['country'].unique()
europe_df = df[df['country'].isin(european)]
print(europe_df.head())
print(europe_df.shape)

     latitude   longitude      country
6   38.063497   32.597950       Turkey
10  46.719055    9.459718  Switzerland
16  55.604940  124.918625       Russia
23  54.434640   36.360918       Russia
27  41.842658   33.732040       Turkey
(2349, 3)


Index column corresponds to image number in the data.

In [18]:
europe_df.to_csv('./Data/dataset/european_images.csv', index=True)